In [41]:
import pandas as pd
import numpy as np
import pytz, sys,os
import plotly.express as px

THIS_DIR = os.path.dirname(__file__)
CODE_DIR = os.abspath(os.join(THIS_DIR, '..', 'code'))
sys.path.append(CODE_DIR)

from datetime import datetime as dt
from datetime import timezone
from dateutil import tz
from src.config import SUBSCRIPTION_KEY, CUSTOMER_ID, get_project_root


root = get_project_root()
os.chdir(root)



NameError: name '__file__' is not defined

In [15]:



root = get_project_root()


from PF_API_Shared import get_all_plants, get_plant_devices, Dispatch,TimeInterval, get_plants_metadata


ModuleNotFoundError: No module named 'src'

In [ ]:

plant = 'Latitude'
start = dt(2022,8,1)
end = dt(2022,9,1)


In [ ]:
dev_tags = [("Combiner", ["DC_CURRENT"]),
            ("Inverter", ["DC_CURRENT"]),
            ("Sensor Pyranometer POA", ["IRRADIANCE_POA"]),
                ("Meter",["AC_POWER"])]
  
rq_tags = []

plants = get_all_plants(SUBSCRIPTION_KEY, CUSTOMER_ID)  
plant_id = plants["id"][plants["name"] == plant].values[0]
dispatch = Dispatch(SUBSCRIPTION_KEY, '2096')
devices = get_plant_devices(SUBSCRIPTION_KEY,CUSTOMER_ID,plant_id)


In [ ]:
for dtype, tags in dev_tags:
    dtype_devices = devices[devices['type']==dtype]
    
    df = dispatch.fetch_data(dtype_devices['id'], 
                                tags, 
                                query_range=TimeInterval(starttime=start,endtime=end))
    rq_tags.append(df)
inv_data = pd.concat(rq_tags, axis=1)
data = inv_data.copy(deep=True)

In [ ]:
ident = [id for site, type, id, name in data.columns]
data.columns = ident
data.tz_convert()

In [ ]:
data = data.set_index([data.index])
data.index = pd.to_datetime(data.index)
data = data[~data.index.duplicated(keep='first')]
data.index = data.index.tz_convert("US/Central")

In [ ]:
"""For Filtered data, the data is filtered by the following parameters
        a. POA > 100
        b. (Meter Power > 1.5 MW) & (Meter Power < 25 MW)
        c. Hour > 12 (For shading purposes)
        d. All CBX's < 0 = 0
"""

data.loc[:, "Meter_Power"] = data.filter(regex="MTR").median(axis=1)
data.loc[:, "POA"] = data.filter(regex="MET").median(axis=1)

curtailment_limit = 499999

new_df = data.between_time("7:00", "17:00")
new_df = data[(data.loc[:,"POA"] > 100)&(data.loc[:,"Meter_Power"]<curtailment_limit*0.99)]
new_df[new_df<=0.00] = 0
day_df = new_df.resample("1D").sum()

day_df


In [11]:
path = "data\\Project_Metadata_tables.xlsx"
print(os.path.abspath(path))

dc_meta = pd.read_excel(path, sheet_name="Combiner_table").filter(items= ['plant_name', 'string_count','DC Rating Power kW', 'Module IMP'], axis=1)


c:\Users\dpinales\Desktop\Arevbot\Dataframe_mangement\data\Project_Metadata_tables.xlsx


FileNotFoundError: [Errno 2] No such file or directory: 'data\\Project_Metadata_tables.xlsx'

In [ ]:
day_df = day_df.filter(regex="CMB")

dc_issues = dc_meta[dc_meta["plant_name"]==plant]  
dc_rating = [i for i in dc_issues['DC Rating Power kW'].values]

relative_dc = []

max_dc = max(dc_rating)

for i in dc_rating:
    dc = (i/max_dc)
    relative_dc.append(dc)

string_count = [s for s in dc_issues['string_count']]

cmb_relative = day_df.div(relative_dc)

cmb_relative.loc[:,'norm'] = cmb_relative.apply(pd.Series.nlargest, axis = 1, n = 5).median(axis = 1)
t = cmb_relative.div(cmb_relative['norm'], axis = 0).clip(lower = 0, upper = 1)
t = t.iloc[:,:-1]

down_strings = (string_count*(1-t)).round()

down_strings.to_csv('latitude_DC.csv')


In [ ]:
e_downstrings = sum(int(down_strings[:]))
total_string = sum(int(string_count))
ratio_strings = e_downstrings/total_string

print(f"The estimated plants DC outage: {ratio_strings}")

In [ ]:
sum_of_cbx = cbx_data["Sum_of_CBX"]
sum_of_inv = inv_df["Sum_of_inv"]

def cmb_summation(a,b):
    total = (a/b)
    return total


In [ ]:
# cbx_data.to_csv(f"{plant} - August_Daily_Derates.csv")
cbx_data.to_csv(f"{plant} - API Data.csv")